# Trabalhando bem

Vamos abordar alguns aspectos de como fazer um algoritmo/pacote
nos padrões esperados do Julia, e também um workflow para desenvolver
um algoritmo rápido, e depois melhorá-lo.

## Um pacote

Uma pacote em Julia, é um diretório com a configuração

 - NomeDoPacote.jl/
   - src/
     - NomeDoPacote.jl
     - Outros arquivos
   - test/
     - runtests.jl
     - Arquivos auxiliares de teste.
   - README.md
   - LICENSE.md
   - .travis.yml
   - Outros arquivos
   
Por mais incomum que seja, o nome do nosso pacote será o nome do diretório,
**incluindo o .jl**. Além disso, esse mesmo nome será o nome do arquivo
principal do pacote.

Os arquivos do `src/` sozinhos não fazem nada. Eles declaram e definem as
funções do pacote. O arquivo `test/` é quem chama o código, assim como
o usuário chamaria. O arquivo `runtests.jl` é o arquivo chamado quando o
usuário fizer `Pkg.test("NomeDoPacote")`.
No índice de pacotes é mostrado se esse teste é bem sucedido.

O arquivo `README.md` é o arquivo principal de informação do seu pacote,
e o arquivo `LICENSE.md` é o arquivo indicando qual a licensa do seu pacote.
A licensa mais utilizada aqui é a MIT, mas outras podem ser utilizadas.

O arquivo `.travis.yml` é um arquivo muito específico que roda os testes
do seu pacote online. Ele exige configuração, e não vamos falar dele agora.

## NomeDoPacote.jl

O arquivo `NomeDoPacote.jl` é o arquivo principal do seu pacote.
A maneira normal de começar é criar um módulo para seu código.

````julia
module NomeDoPacote

...

end
````

Dentro desse módulo, normalmente incluímos códigos auxiliares. Por exemplo, vamos colocar a busca linear num
arquivo separado `buscaLinear.jl`. Então colocamos `include("buscaLinear.jl")`.

Agora criamos a nossa função `metodo_gradiente` e colocamos um comando
`export metodo_gradiente` para que usuários possam usar essa função.

Agora criamos algum teste para nosso código. Vamos criar o arquivo
`teste_quad.jl` contendo testes de funções quadráticas, e o no arquivo
`runtests.jl` colocamos `include("teste_quad.jl")`.

Veja o MetodoGradiente.jl.

In [4]:
;ls -R MetodoGradiente.jl

MetodoGradiente.jl:
src
test

MetodoGradiente.jl/src:
BuscaLinear.jl
MetodoGradiente.jl

MetodoGradiente.jl/test:
runtests.jl
test_quad.jl


## Workflow de um código de otimização

Até bem pouco tempo, tínhamos duas opções de implementações
de algoritmos de otimização

 - C/Fortran: Um algoritmo rápido, competitivo, porém trabalhoso de fazer e usar;
 - MatLab: Um algoritmo fácil de escrever, porém limitado em alguns aspectos de velocidade e liberdade de uso.

O Python criou uma alternativa que não fui suficientemente adotada por não ser fácil o suficiente,
mas com Julia, temos uma oportunidade nova.

O Julia permite uma implementação rápida, é livre, e tem uma velocidade boa.
Além disse, permite uma integração com C e Fortran muito maior, que pode ser feita gradualmente.
Eventualmente, se necessário, pode-se fazer uma migração total.

Esse paradigma funciona para quase qualquer tipo de aplicação em Julia,
mas para otimização vamos ver como o CUTEst pode nos ajudar.

### Ponto inicial

A primeira preocupação deve ser fazer o algoritmo funcionar.

 - Não se preocupe em deixar o código eficiente;
 - Não se preocupe com sistemas lineares;
 - Não se preocupe com BLAS
 - Não se preocupe com variáveis repetidas;
 - Não se preocupe com matrizes criadas à toa;

Palavras do Donald Knuth:

> Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: **premature optimization is the root of all evil.** Yet we should not pass up our opportunities in that critical 3%.
> - Structured Programming With Go To Statements", Computing Surveys, Vol 6, No 4, December 1974 

Escreva os testes, decida o que o código precisa resolver e implemente tudo.

 - Use chamadas de função explícitas;
 - Use `\` para resolver sistemas;

In [33]:
function metodo_bfgs(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    B = eye(length(x₀))
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -B\∇f(x)
        dt∇f = dot(d,∇f(x))
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dt∇f
            t *= η
        end
        s = t*d
        y = ∇f(x+s)- ∇f(x)
        x = x + s
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        if dot(s,y) > 0.0
            B = B + y*y'/dot(y,s) - B*s*s'*B/dot(s,B*s)
        end
    end
    return x, f(x), ef, k
end

metodo_bfgs (generic function with 1 method)

### CUTEst

Seu código funciona? Teste com o CUTEst. Neste caso você pode usar as funções de nível usuário final ou nível intermediário.

In [34]:
# Nível usuário final
using CUTEst
nlp = CUTEstModel("ROSENBR")
f(x) = obj(nlp, x)
∇f(x) = grad(nlp, x)

x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")
cutest_finalize(nlp)

x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34


In [35]:
# Nível intermediário
using CUTEst
nlp = CUTEstModel("ROSENBR")
f(x) = ufn(nlp, x)
∇f(x) = ugr(nlp, x)

x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")
cutest_finalize(nlp)

#### Rodando para uma lista de problemas

In [36]:
lista = ["ROSENBR", "BARD", "ALLINITU", "CUBE", "BEALE", "OSCIPATH"]

for p in lista
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
    println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    cutest_finalize(nlp)
end

x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34
x = [0.08241305128208429,1.1331146993324175,2.343619736562304], f(x) = 0.008214877350822282; ef = 0; k = 18
x = [1.459861381315666,0.0,0.08089056947684974,-0.8111127629456985], f(x) = 5.744384910322372; ef = 0; k = 13
x = [0.9999993508975884,0.9999980594457627], f(x) = 4.2589253127599017e-13; ef = 0; k = 55
x = [2.9999885924594127,0.4999972321719778], f(x) = 2.091480518781361e-11; ef = 0; k = 12


### Melhorias

Agora que temos um código e sabemos que ele funciona para alguns problemas (a lista pode ser maior), vamos fazer algumas melhorias.

#### Exception Handling

Quase toda linguagem atual tem alguma maneira de lidar com exceções.
Como `nlp` precisa ser finalizado, podemos fazer o seguinte

In [37]:
for p in lista
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    try
        x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0)
        println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    catch er
        println("ERRO no problema $p:", er)
    finally
        cutest_finalize(nlp)
    end
end

x = [-0.9999333281751371,0.9999833321003804,0.9999958326256179,0.9999989565193412,0.9999997325791267,0.9999999069417307,0.9999998719228206,0.9999995487242608,0.9999982101015056,0.9999928439885907], f(x) = 0.9999666654928699; ef = 0; k = 11
x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34
x = [0.08241305128208429,1.1331146993324175,2.343619736562304], f(x) = 0.008214877350822282; ef = 0; k = 18
x = [1.4598611392227534,0.0,0.08089124534078221,-0.8111123290657675], f(x) = 5.744384910331394; ef = 0; k = 12
x = [0.9999896122707613,0.9999689465124421], f(x) = 1.0910123939612766e-10; ef = 0; k = 51
x = [2.9999885924594127,0.4999972321719778], f(x) = 2.091480518781361e-11; ef = 0; k = 12


#### Reaproveitar

 - Chamar `f(x)` e $\nabla$`f(x)` toda vez é ruim.
 - Produtos internos
 - Bs = B*s

In [38]:
function metodo_bfgs2(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    B = eye(length(x₀))
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = ∇f(x)
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        y = ∇fx # Não copia, só passa o ponteiro
        fx = f(x)
        ∇fx = ∇f(x)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        sty = dot(s,y)
        if sty > 0.0
            Bs = B*s
            B = B + y*y'/sty - Bs*Bs'/dot(s,Bs)
        end
    end
    return x, fx, ef, k
end

metodo_bfgs2 (generic function with 1 method)

In [39]:
for p in lista
    nlp = CUTEstModel(p)
    f(x) = ufn(nlp, x)
    ∇f(x) = ugr(nlp, x)
    try
        x, fx, ef, k = metodo_bfgs2(f, ∇f, nlp.meta.x0)
        println("x = $x, f(x) = $fx; ef = $ef; k = $k")
    catch er
        println("ERRO no problema $p:", er)
    finally
        cutest_finalize(nlp)
    end
end

x = [-0.9999333281751371,0.9999833321003804,0.9999958326256179,0.9999989565193412,0.9999997325791267,0.9999999069417307,0.9999998719228206,0.9999995487242608,0.9999982101015056,0.9999928439885907], f(x) = 0.9999666654928699; ef = 0; k = 11
x = [0.9999996948881872,0.9999993862917482], f(x) = 9.43075451728985e-14; ef = 0; k = 34
x = [0.08241305128208423,1.1331146993324166,2.343619736562305], f(x) = 0.008214877350822271; ef = 0; k = 18
x = [1.4598611392227536,0.0,0.08089124534078224,-0.8111123290657679], f(x) = 5.744384910331395; ef = 0; k = 12
x = [0.9999896127140547,0.9999689476404822], f(x) = 1.0908761933602825e-10; ef = 0; k = 51
x = [2.999988592459412,0.49999723217197745], f(x) = 2.0914805191053063e-11; ef = 0; k = 12


In [40]:
function roda_cutest(mtd; T = 1000)
    lista = ["ROSENBR", "BARD", "ALLINITU", "CUBE", "BEALE", "OSCIPATH"]
    tempos = zeros(length(lista))
    for (i,p) in enumerate(lista)
        nlp = CUTEstModel(p)
        f(x) = ufn(nlp, x)
        ∇f(x) = ugr(nlp, x)
        try
            x, fx, ef, k = mtd(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
            for j = 1:T
                x, fx, ef, k = mtd(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=1000)
            end
        catch er
            println("ERRO no problema $p:", er)
        finally
            cutest_finalize(nlp)
        end
    end
end

roda_cutest (generic function with 1 method)

In [41]:
for mtd in [metodo_bfgs, metodo_bfgs2]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd)
end

x = [-0.9999333281751371,0.9999833321003804,0.9999958326256179,0.9999989565193412,0.9999997325791267,0.9999999069417307,0.9999998719228206,0.9999995487242608,0.9999982101015056,0.9999928439885907], f(x) = 0.9999666654928699; ef = 0; k = 11
  2.209770 seconds (16.94 M allocations: 975.564 MB, 6.46% gc time)
  

#### Melhorias mais sérias

 - `B\g`: Resolver o sistema assim não é uma boa. No entanto, se o sistema for
  pequeno, e a matriz for densa, não faz quase diferença nenhuma. Então mantemos.
 - Atualização do BFGS: Cara de fazer (mas o problema é a matriz existir mesmo)

In [42]:
function metodo_bfgs3(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    n = length(x₀)
    B = eye(n)
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = ∇f(x)
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        y = ∇fx # Não copia, só passa o ponteiro
        fx = f(x)
        ∇fx = ∇f(x)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        sty = dot(s,y)
        if sty > 0.0
            Bs = B*s
            stBs = dot(s,Bs)
            for i = 1:n
                B[i,i] += y[i]^2/sty - Bs[i]^2/stBs
                for j = i+1:n
                    B[i,j] = B[i,j] + y[i]*y[j]/sty - Bs[i]*Bs[j]/stBs
                    B[j,i] = B[i,j]
                end
            end
        end
    end
    return x, fx, ef, k
end

metodo_bfgs3 (generic function with 1 method)

In [60]:
for mtd in [metodo_bfgs, metodo_bfgs2, metodo_bfgs3]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd)
end

  2.207105 seconds (16.94 M allocations: 975.562 MB, 6.64% gc time)
  1.435548 seconds (12.37 M allocations: 706.143 MB, 7.42% gc time)
  1.501740 seconds (22.43 M allocations: 668.264 MB, 5.95% gc time)


### Inplace

Como dissemos, o CUTEst tem várias interfaces. Uma que ajuda bastante é a interface *inplace*.

In [44]:
?ugr

# ugr

The ugr subroutine evaluates the gradient of the objective function of the problem decoded from a SIF file by the script sifdecoder at the point X. The problem under consideration is to minimize or maximize an objective function f(x) over all x ∈ Rn subject to the simple bounds xl≤x≤xu. The objective function is group-partially separable.

This help was generated automatically and may contain errors. For more information, run the shell command

```
man cutest_ugr
```

Usage:

```
ugr(io_err, n, x, g)
```

  * io_err:  [OUT] Array{Cint, 1}
  * n:       [IN] Array{Cint, 1}
  * x:       [IN] Array{Cdouble, 1}
  * g:       [OUT] Array{Cdouble, 1}

```
g = ugr(n, x)
```

  * n:       [IN] Int
  * x:       [IN] Array{Float64, 1}
  * g:       [OUT] Array{Float64, 1}

```
ugr!(n, x, g)
```

  * n:       [IN] Int
  * x:       [IN] Array{Float64, 1}
  * g:       [OUT] Array{Float64, 1}

```
g = ugr(nlp, x)
```

  * nlp:     [IN] CUTEstModel
  * x:       [IN] Array{Float64, 1}
  * g:       [OUT] Array{Float64, 1}

```
ugr!(nlp, x, g)
```

  * nlp:     [IN] CUTEstModel
  * x:       [IN] Array{Float64, 1}
  * g:       [OUT] Array{Float64, 1}


In [45]:
nlp = CUTEstModel("ROSENBR")
g = zeros(nlp.meta.nvar)
ugr!(nlp, nlp.meta.x0, g)
println("g = $g")
println(ugr(nlp, nlp.meta.x0))
cutest_finalize(nlp)

Muito mais chato de usar, mas você reutiliza o vetor criado

In [46]:
# Atenção: ∇f!(x, g).
function metodo_bfgs4(f, ∇f!, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    n = length(x₀)
    B = eye(n)
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = zeros(n)
    ∇f!(x, ∇fx)
    y = zeros(n) # Agora criamos y aqui. Uma vez
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        copy!(y, ∇fx) # Sobrescreve os valores de y
        fx = f(x)
        ∇f!(x, ∇fx)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        sty = dot(s,y)
        if sty > 0.0
            Bs = B*s
            stBs = dot(s,Bs)
            for i = 1:n
                B[i,i] += y[i]^2/sty - Bs[i]^2/stBs
                for j = i+1:n
                    B[i,j] = B[i,j] + y[i]*y[j]/sty - Bs[i]*Bs[j]/stBs
                    B[j,i] = B[i,j]
                end
            end
        end
    end
    return x, fx, ef, k
end

metodo_bfgs4 (generic function with 1 method)

In [47]:
function roda_cutest_inplace(mtd; T = 1000)
    lista = ["ROSENBR", "BARD", "ALLINITU", "CUBE", "BEALE", "OSCIPATH"]
    tempos = zeros(length(lista))
    for (i,p) in enumerate(lista)
        nlp = CUTEstModel(p)
        f(x) = ufn(nlp, x)
        ∇f!(x, g) = ugr!(nlp, x, g)
        try
            x, fx, ef, k = mtd(f, ∇f!, nlp.meta.x0, ϵ=1e-5, kmax=1000)
            for j = 1:T
                x, fx, ef, k = mtd(f, ∇f!, nlp.meta.x0, ϵ=1e-5, kmax=1000)
            end
        catch er
            println("ERRO no problema $p:", er)
        finally
            cutest_finalize(nlp)
        end
    end
end

roda_cutest_inplace (generic function with 1 method)

1.565220 seconds (22.43 M allocations: 668.264 MB, 5.58% gc time)
search: ugr ugr! ugrsh ugreh ugrdh ugrsh! ugreh! ugrdh! ClusterManager

g = [-215.59999999999997,-87.99999999999999]
[-215.59999999999997,-87.99999999999999]


In [48]:
for mtd in [metodo_bfgs, metodo_bfgs2, metodo_bfgs3]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd)
end
for mtd in [metodo_bfgs4]
    roda_cutest_inplace(mtd, T=1)
    @time roda_cutest_inplace(mtd)
end

  2.232657 seconds (16.94 M allocations: 975.562 MB, 6.55% gc time)
  1.468368 seconds (12.37 M allocations: 706.144 MB, 7.45% gc time)
  1.554630 seconds (22.43 M allocations: 668.264 MB, 5.69% gc time)
  

Mudar o código para usar inplace pode levar à uma grande melhoria, mas é trabalhoso e quebra o código.
Faça quando tiver tempo.

## Baixar o nível do código?

 - O que mandar pra C? Busque pelo profiler.
 - A mudança é necessária? Alternativas?
 - A mudança é viável?
 
No nosso caso, poderíamos tentar mudar o BFGS por um BFGS explícito, com memória limitada.
Poderíamos usar uma busca melhor. Poderíamos usar região de confiança, etc.

In [49]:
;cat src/update_bfgs.c

In [50]:
;gcc src/update_bfgs.c -o update_bfgs.o -fPIC -shared

In [51]:
B = eye(3)
y = ones(3)
s = rand(3)
B + y*y'/dot(s,y) - B*s*s'*B/dot(s,B*s)

3x3 Array{Float64,2}:
 1.42233   0.283961  0.34564
 0.283961  0.987426  0.11961
 0.34564   0.11961   1.22036

In [52]:
println("B = \n$B")
ccall( (:update_bfgs, "update_bfgs.o"), # (foo, lib)
  Void, # return
  (Cint, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}), # (type, of, each, argument)
Cint(3), B, y, s) # each, argument
println("B = \n$B")

In [53]:
function metodo_bfgs5(f, ∇f!, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    n = length(x₀)
    B = eye(n)
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = zeros(n)
    ∇f!(x, ∇fx)
    y = zeros(n) # Agora criamos y aqui. Uma vez
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        copy!(y, ∇fx) # Sobrescreve os valores de y
        fx = f(x)
        ∇f!(x, ∇fx)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        ccall( (:update_bfgs, "update_bfgs.o"), Void,
            (Cint, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}),
            Cint(n), B, y, s)
    end
    return x, fx, ef, k
end

metodo_bfgs5 (generic function with 1 method)

In [54]:
for mtd in [metodo_bfgs, metodo_bfgs2, metodo_bfgs3]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd)
end
for mtd in [metodo_bfgs4, metodo_bfgs5]
    roda_cutest_inplace(mtd, T=1)
    @time roda_cutest_inplace(mtd)
end

1.181830 seconds (7.70 M allocations: 433.990 MB, 5.34% gc time)
#include <stdio.h>
#include <stdlib.h>

void update_bfgs(int n, double *B, double *y, double *s) {
  int i, j, k;
  double *Bs, a = 0.0, b = 0.0;

  for (i = 0; i < n; i++)
    a += y[i]*s[i];

  if (a <= 0)
    return;

  Bs = (double *) malloc(sizeof(double)*n);

  for (i = 0; i < n; i++) {
    Bs[i] = 0.0;
    for (j = 0; j < n; j++)
      Bs[i] += B[i + j*n]*s[j];
    b += s[i]*Bs[i];
  }

  for (i = 0; i < n; i++) {
    for (j = i; j < n; j++) {
      B[i + j*n] += y[i]*y[j]/a - Bs[i]*Bs[j]/b;
      B[j + i*n] = B[i + j*n];
    }
  }

  free(Bs);

}

void update_bfgs2(int n, double *B, double *y, double *s, double *Bs) {
  int i, j, k;
  double a = 0.0, b = 0.0;

  for (i = 0; i < n; i++)
    a += y[i]*s[i];

  if (a <= 0)
    return;

  for (i = 0; i < n; i++) {
    Bs[i] = 0.0;
    for (j = 0; j < n; j++)
      Bs[i] += B[i + j*n]*s[j];
    b += s[i]*Bs[i];
  }

  for (i = 0; i < n; i++) {
    for (j = i; j < n; j+

In [55]:
# Atenção: ∇f!(x, g).
function metodo_bfgs6(f, ∇f!, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    n = length(x₀)
    B = eye(n)
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = zeros(n)
    ∇f!(x, ∇fx)
    y = zeros(n) # Agora criamos y aqui. Uma vez
    Bs = zeros(n)
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        copy!(y, ∇fx) # Sobrescreve os valores de y
        fx = f(x)
        ∇f!(x, ∇fx)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        sty = dot(s,y)
        if sty > 0.0
            for i = 1:n
                Bs[i] = 0.0
                for j = 1:n
                    Bs[i] += B[i,j]*s[j]
                end
            end
            
            stBs = dot(s,Bs)
            for i = 1:n
                B[i,i] += y[i]^2/sty - Bs[i]^2/stBs
                for j = i+1:n
                    B[i,j] = B[i,j] + y[i]*y[j]/sty - Bs[i]*Bs[j]/stBs
                    B[j,i] = B[i,j]
                end
            end
        end
    end
    return x, fx, ef, k
end

metodo_bfgs6 (generic function with 1 method)

In [56]:
for mtd in [metodo_bfgs, metodo_bfgs2, metodo_bfgs3]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd)
end
for mtd in [metodo_bfgs4, metodo_bfgs5, metodo_bfgs6]
    roda_cutest_inplace(mtd, T=1)
    @time roda_cutest_inplace(mtd)
end

1.171909 seconds (7.41 M allocations: 419.740 MB, 5.23% gc time)
  2.243280 seconds (16.94 M allocations: 975.559 MB, 6.79% gc time)
  1.467551 seconds (12.37 M allocations: 706.145 MB, 7.39% gc time)
  1.545693 seconds (22.43 M allocations: 668.262 MB, 5.51% gc time)
  1.180867 seconds (7.70 M allocations: 433.990 MB, 5.38% gc time)
  1.168218 seconds (7.41 M allocations: 419.740 MB, 5.23% gc time)
  

In [57]:
function metodo_bfgs7(f, ∇f!, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    n = length(x₀)
    B = eye(n)
    ef = 0
    x = copy(x₀)
    fx = f(x)
    ∇fx = zeros(n)
    ∇f!(x, ∇fx)
    y = zeros(n)
    Bs = zeros(n)
    k = 0
    while norm(∇fx) > ϵ
        d = -B\∇fx
        dt∇f = dot(d,∇fx)
        t = 1.0
        while f(x + t*d) > fx + α*t*dt∇f
            t *= η
        end
        s = t*d
        x = x + s
        copy!(y, ∇fx) # Sobrescreve os valores de y
        fx = f(x)
        ∇f!(x, ∇fx)
        y = ∇fx - y
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        ccall( (:update_bfgs2, "update_bfgs.o"), Void,
          (Cint, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}, Ptr{Cdouble}),
          Cint(n), B, y, s, Bs)
    end
    return x, fx, ef, k
end

metodo_bfgs7 (generic function with 1 method)

In [59]:
T = 2000
for mtd in [metodo_bfgs, metodo_bfgs2, metodo_bfgs3]
    roda_cutest(mtd, T=1)
    @time roda_cutest(mtd, T=T)
end
for mtd in [metodo_bfgs4, metodo_bfgs5, metodo_bfgs6, metodo_bfgs7]
    roda_cutest_inplace(mtd, T=1)
    @time roda_cutest_inplace(mtd, T=T)
end

  4.078675 seconds (33.86 M allocations: 1.903 GB, 7.47% gc time)
  2.516413 seconds (24.72 M allocations: 1.377 GB, 8.77% gc time)
  2.684240 seconds (44.84 M allocations: 1.303 GB, 7.03% gc time)
  1.961218 seconds (15.38 M allocations: 865.906 MB, 6.82% gc time)
  1.924396 seconds (14.81 M allocations: 837.419 MB, 6.51% gc time)
  1.920262 seconds (14.82 M allocations: 838.515 MB, 6.83% gc time)
  1.904263 seconds (14.82 M allocations: 838.518 MB, 6.54% gc time)
